## Multi-Label Classifier with Feed-Forward Nural Netwark

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import pandas as pd
import numpy as np

In [3]:
CICIDS_data = pd.read_csv('../Wednesday-workingHours.pcap_ISCX.csv')

In [4]:
# dropping duplications
def find_duplicated_columns(df):
    grouped_columns = []
    seen_columns = set()
    for col in df.columns:
        if col not in seen_columns:
            duplicates = [col] + [c for c in df.columns if c != col and df[col].equals(df[c])]
            if len(duplicates)>1:
                grouped_columns.extend(duplicates[1:])
            seen_columns.update(duplicates)
    return grouped_columns

In [5]:
# data preproccessing 
CICIDS_data.columns = [col.strip() for col in CICIDS_data.columns]
CICIDS_data.drop(CICIDS_data.columns[CICIDS_data.eq(0).all()], axis=1, inplace=True)
CICIDS_data.drop(find_duplicated_columns(CICIDS_data), axis=1, inplace=True)

In [6]:
# splitting my data into features (X) and target (y) ignoring the columns with inf values
X = CICIDS_data.drop(['Label','Flow Bytes/s', 'Flow Packets/s'],axis=1).copy()
y = CICIDS_data['Label'].copy()

In [7]:
# filling nan values with -1
X.fillna(-1, inplace=True)

In [8]:
# scaling my features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# encoding my categorical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [10]:
# splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [11]:
# building the model with 3 hidden layers
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

2024-07-18 13:59:12.074191: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# compailing the model with hyper parameters
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
# training the model with the training set
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

2024-07-18 13:59:12.213725: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
13855/13855 [==============================] - 11s 769us/step - loss: 742.4066 - accuracy: 0.9426 - val_loss: 0.3756 - val_accuracy: 0.9584
Epoch 2/20
13855/13855 [==============================] - 11s 811us/step - loss: 0.6252 - accuracy: 0.9612 - val_loss: 0.1929 - val_accuracy: 0.9639
Epoch 3/20
13855/13855 [==============================] - 10s 726us/step - loss: 0.2485 - accuracy: 0.9638 - val_loss: 0.1544 - val_accuracy: 0.9626
Epoch 4/20
13855/13855 [==============================] - 11s 775us/step - loss: 0.4285 - accuracy: 0.9649 - val_loss: 0.1485 - val_accuracy: 0.9632
Epoch 5/20
13855/13855 [==============================] - 12s 841us/step - loss: 0.1621 - accuracy: 0.9647 - val_loss: 0.1245 - val_accuracy: 0.9664
Epoch 6/20
13855/13855 [==============================] - 10s 754us/step - loss: 1.6649 - accuracy: 0.9669 - val_loss: 0.1360 - val_accuracy: 0.9697
Epoch 7/20
13855/13855 [==============================] - 16s 1ms/step - loss: 0.2117 - accuracy: 0.9686

In [14]:
# predicting the classes in the test set
y_pred = np.argmax(model.predict(X_test), axis=1)

In [15]:
# evaluating the model performance 
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print(confusion_matrix(y_test, y_pred))

                  precision    recall  f1-score   support

          BENIGN       0.95      1.00      0.97     88170
   DoS GoldenEye       1.00      0.58      0.74      2017
        DoS Hulk       1.00      0.96      0.98     46147
DoS Slowhttptest       1.00      0.07      0.14      1090
   DoS slowloris       1.00      0.02      0.04      1114
      Heartbleed       0.00      0.00      0.00         3

        accuracy                           0.96    138541
       macro avg       0.82      0.44      0.48    138541
    weighted avg       0.97      0.96      0.96    138541

[[88078     1    91     0     0     0]
 [  836  1179     2     0     0     0]
 [ 2005     1 44141     0     0     0]
 [ 1009     0     0    81     0     0]
 [ 1092     0     1     0    21     0]
 [    3     0     0     0     0     0]]


/Users/traviv/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/traviv/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/traviv/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
